In [1]:
import numpy as np
import time
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [2]:
# PID parameters
kp = 1
ki = 0.8
kd = 0.01

In [3]:
# Initialize
processval = 0.0
controlvar = 0.0
setpoint = 0.0
error = 0.0
prop_term = 0.0
inte_term = 0.0
deri_term = 0.0
lasterr = 0.0
samplingtime = 0.1  # seconds
currenttime = time.time()
lasttime = currenttime
maxsetpoint = 1.0
winduplim = maxsetpoint / 5

In [4]:
# Setup plots
timesteps = 50
timepoints = []
outputs = []
setpoints = []
errors = []
p1 = figure(plot_width=400, plot_height=300, x_range=[0, timesteps * samplingtime],
          y_range=[-2 * maxsetpoint, 2 * maxsetpoint], title='PID controller input and output')
p2 = figure(plot_width=400, plot_height=300, x_range=[0, timesteps * samplingtime],
          y_range=[-2 * maxsetpoint, 2 * maxsetpoint], title='PID controller error')
r1 = p1.line(timepoints, setpoints, line_color='cornflowerblue', legend='input',
            line_width=2)
r2 = p1.line(timepoints, outputs, line_color='indianred', legend='output',
            line_width=2)
r3 = p2.line(timepoints, errors, line_color='indigo', legend='error',
            line_width=2)
target = show(row(p1, p2), notebook_handle=True)
p1.legend.location = 'bottom_center'  
p2.legend.location = 'bottom_center'  

# PID control loop
for nn in range(1, timesteps):
    
    # Inputs
#     if nn < 10:
#         setpoint = maxsetpoint * nn / 10
#     else:
#         setpoint = maxsetpoint
#     if nn < 10:
#         setpoint = 2.0 * maxsetpoint * nn / 10
#     elif (nn >= 10) and (nn < 20):
#         setpoint = 2.0 * maxsetpoint - maxsetpoint * (nn - 10) / 10
#     else:
#         setpoint = maxsetpoint
#     if nn > timesteps / 2:
#         setpoint = maxsetpoint
    setpoint = -maxsetpoint * np.sign(np.cos(2 * np.pi * nn / timesteps))
#     setpoint = maxsetpoint * np.sin(4 * np.pi * nn / timesteps)
    processval = processval + (controlvar - (1.0 / nn))
    outputs.append(processval)
    timepoints.append(nn * samplingtime)
    setpoints.append(setpoint)
    errors.append(error)
    
    # PID control update
    time.sleep(samplingtime)
    error = setpoint - processval
    currenttime = time.time()
    dt = currenttime - lasttime
    if dt > samplingtime:
        prop_term = kp * error
        inte_term = inte_term + ki * error * dt
    if inte_term < -winduplim:
        inte_term = -winduplim
    elif inte_term > winduplim:
        inte_term = winduplim
    deri_term = kd * (error - lasterr) / dt
    controlvar = prop_term + inte_term + deri_term
    lasterr = error
    lasttime = currenttime
    
    # Update plots
    r1.data_source.data['x'] = timepoints
    r1.data_source.data['y'] = setpoints
    r2.data_source.data['x'] = timepoints
    r2.data_source.data['y'] = outputs
    r3.data_source.data['x'] = timepoints
    r3.data_source.data['y'] = errors
    push_notebook(handle=target)